## 1. Setup & Import

Đầu tiên, cần thiết lập đường dẫn để import module `src` từ thư mục `23127011`.

In [1]:
import sys
import os

# Xác định đường dẫn project
# Notebook nằm trong: Milestone2_Project/23127011/notebooks/
# Src nằm trong: Milestone2_Project/23127011/src/

NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(NOTEBOOK_DIR)  # 23127011
SRC_PARENT = PROJECT_ROOT  # Thư mục chứa src

# Thêm vào sys.path để import được
if SRC_PARENT not in sys.path:
    sys.path.insert(0, SRC_PARENT)

print(f"📂 Project Root: {PROJECT_ROOT}")
print(f"📂 Src Parent:   {SRC_PARENT}")

📂 Project Root: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011
📂 Src Parent:   d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011


In [2]:
# Import các module cần thiết
from src import (
    run_full_pipeline,          # Chạy toàn bộ pipeline
    run_processing_pipeline,    # Chỉ Phase 1
    run_matching_pipeline,      # Chỉ Phase 2
)
from src.config import PipelineConfig, create_config

print("✅ Import thành công!")

✅ Import thành công!


---
## 2. Cấu hình Pipeline

Thiết lập các đường dẫn input/output và các tham số cấu hình.

In [ ]:
# === CẤU HÌNH ĐƯỜNG DẪN ===

# Input: Thư mục chứa dữ liệu LaTeX thô
# Có thể dùng 'data_raw_test' để test với dataset nhỏ
DATA_RAW = os.path.join(PROJECT_ROOT, '../', 'data_raw_test')

# Output: Thư mục xuất kết quả
DATA_OUTPUT = os.path.join(PROJECT_ROOT, 'data_output_tutorial')

# === CẤU HÌNH XỬ LÝ ===

# Sử dụng xử lý song song để tăng tốc
USE_PARALLEL = True

# Số luồng (None = auto-detect từ số CPU)
MAX_WORKERS = os.cpu_count() or 4

print(f"📂 Data Raw:   {DATA_RAW}")
print(f"📂 Data Output: {DATA_OUTPUT}")
print(f"⚙️  Parallel:    {USE_PARALLEL}")
print(f"⚙️  Workers:     {MAX_WORKERS}")

📂 Data Raw:   d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\../data_raw_test
📂 Data Output: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\data_output_tutorial
⚙️  Parallel:    True
⚙️  Workers:     20


In [4]:
# Kiểm tra data_raw tồn tại và có bao nhiêu papers
if os.path.exists(DATA_RAW):
    papers = [f for f in os.listdir(DATA_RAW) 
              if os.path.isdir(os.path.join(DATA_RAW, f))]
    print(f"✅ Tìm thấy {len(papers)} papers trong data_raw")
    if papers:
        print(f"   Ví dụ: {papers[:5]}")
else:
    print(f"❌ Không tìm thấy thư mục: {DATA_RAW}")
    print("   Hãy đảm bảo đường dẫn DATA_RAW chính xác!")

✅ Tìm thấy 11 papers trong data_raw
   Ví dụ: ['2403-00530', '2403-00531', '2403-00532', '2403-00533', '2403-00534']


---
## 3. Phase 1: Pre-processing & Parsing

Phase này thực hiện:
1. **Flatten LaTeX**: Gộp `\input`, `\include` thành 1 file
2. **Extract References**: Trích xuất từ `.bbl`, `.bib`, `\bibitem`
3. **Dedup References**: Loại bỏ reference trùng lặp
4. **Build Structure Tree**: Xây dựng cây Section/Subsection
5. **Process Content**: Làm sạch và tách câu
6. **Export**: `hierarchy.json`, `refs.bib`

In [5]:
# Chạy Phase 1
print("🚀 Bắt đầu Phase 1: Pre-processing & Parsing...")
print(f"   Log file: {os.path.join(DATA_OUTPUT, 'pipeline.log')}")
print()

run_processing_pipeline(
    data_raw_path=DATA_RAW,
    data_output_path=DATA_OUTPUT,
    parallel=USE_PARALLEL,
    max_workers=MAX_WORKERS
)

print("\n✅ Phase 1 hoàn thành!")

🚀 Bắt đầu Phase 1: Pre-processing & Parsing...
   Log file: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\data_output_tutorial\pipeline.log


✅ Phase 1 hoàn thành!


In [6]:
# Kiểm tra kết quả Phase 1
if os.path.exists(DATA_OUTPUT):
    output_folders = os.listdir(DATA_OUTPUT)
    # Lọc bỏ file log
    output_folders = [f for f in output_folders 
                      if os.path.isdir(os.path.join(DATA_OUTPUT, f))]
    
    print(f"📊 Đã xử lý: {len(output_folders)} papers")
    
    if output_folders:
        # Xem nội dung 1 paper mẫu
        sample = output_folders[0]
        sample_path = os.path.join(DATA_OUTPUT, sample)
        print(f"\n📁 Nội dung của '{sample}':")
        for f in os.listdir(sample_path):
            fpath = os.path.join(sample_path, f)
            size = os.path.getsize(fpath)
            print(f"   - {f} ({size:,} bytes)")

📊 Đã xử lý: 11 papers

📁 Nội dung của '2403-00530':
   - hierarchy.json (166,426 bytes)
   - metadata.json (462 bytes)
   - references.json (31,668 bytes)
   - refs.bib (97,894 bytes)


---
## 4. Phase 2: Reference Matching

Phase này thực hiện:
1. **Load Ground Truth**: Đọc `references.json` (từ arXiv API)
2. **Load Extracted Refs**: Đọc `refs.bib` từ Phase 1
3. **TF-IDF Matching**: So khớp bằng cosine similarity
4. **Export**: `labels.json`

In [7]:
# Chạy Phase 2
print("🔍 Bắt đầu Phase 2: Reference Matching...")
print()

run_matching_pipeline(DATA_OUTPUT)

print("\n✅ Phase 2 hoàn thành!")

🔍 Bắt đầu Phase 2: Reference Matching...

🚀 Starting Matching Pipeline (Phase 2.2)...
   Target: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\data_output_tutorial


Matching References: 100%|██████████| 11/11 [00:01<00:00,  7.16it/s]

✅ Matching Complete. 'labels.json' generated in all folders.

✅ Phase 2 hoàn thành!


---
## 5. Kiểm tra kết quả

Xem chi tiết các file output được tạo ra.

In [8]:
import json

# Chọn một paper để xem chi tiết
if os.path.exists(DATA_OUTPUT):
    folders = [f for f in os.listdir(DATA_OUTPUT) 
               if os.path.isdir(os.path.join(DATA_OUTPUT, f))]
    
    if folders:
        sample_paper = folders[0]
        sample_path = os.path.join(DATA_OUTPUT, sample_paper)
        
        print(f"📄 Chi tiết paper: {sample_paper}")
        print("=" * 50)

📄 Chi tiết paper: 2403-00530


In [9]:
# Xem hierarchy.json (cấu trúc document)
hierarchy_path = os.path.join(sample_path, 'hierarchy.json')

if os.path.exists(hierarchy_path):
    with open(hierarchy_path, 'r', encoding='utf-8') as f:
        hierarchy = json.load(f)
    
    print("📊 hierarchy.json:")
    print(f"   - Versions: {list(hierarchy.get('hierarchy', {}).keys())}")
    print(f"   - Elements: {len(hierarchy.get('elements', {}))} unique items")
    
    # Xem vài element mẫu
    elements = hierarchy.get('elements', {})
    sample_keys = list(elements.keys())[:3]
    print("\n   Sample elements:")
    for k in sample_keys:
        content = elements[k][:100] + "..." if len(elements[k]) > 100 else elements[k]
        print(f"   [{k}]: {content}")

📊 hierarchy.json:
   - Versions: ['1', '2']
   - Elements: 360 unique items

   Sample elements:
   [2403-00530-2403-00530v1-document-b11dd971-36d1-42cf-a133-b0d34b600614]: Root Document
   [2403-00530-2403-00530v1-title-ffcee407-17c2-4405-bc53-10d2b007d6a1]: Extended Hubbard corrected tight-binding model for rhombohedral few-layer graphene
   [2403-00530-2403-00530v1-authors-bd9f9e87-8d89-49d8-a40f-16aafb875681]: Dongkyu Lee, Wooil Yang, Young-Woo Son, Jeil Jung


In [10]:
# Xem refs.bib (references đã dedup)
refs_path = os.path.join(sample_path, 'refs.bib')

if os.path.exists(refs_path):
    with open(refs_path, 'r', encoding='utf-8') as f:
        refs_content = f.read()
    
    # Đếm số refs
    num_refs = refs_content.count('@misc')
    print(f"📚 refs.bib: {num_refs} unique references")
    
    # Xem 1 entry mẫu
    print("\n   Sample entry:")
    first_entry = refs_content.split('@misc')[1] if num_refs > 0 else ""
    print(f"   @misc{first_entry[:300]}...")

📚 refs.bib: 66 unique references

   Sample entry:
   @misc{ref_0,
  text = {@article{Anisimov1991PRB,
  pages = {943--954},
  year = {1991},
  month = {July},
  author = {Anisimov, Vladimir I. and Zaanen, Jan and Andersen, Ole K.},
  journal = {Physical Review B},
  urldate = {2024-01-29},
  number = {3},
  abstract = {We propose a form for the exchange-co...


In [11]:
# Xem labels.json (kết quả matching)
labels_path = os.path.join(sample_path, 'labels.json')

if os.path.exists(labels_path):
    with open(labels_path, 'r', encoding='utf-8') as f:
        labels = json.load(f)
    
    print(f"🏷️  labels.json: {len(labels)} matched references")
    
    if labels:
        # Xem 1 label mẫu
        sample_label = labels[0]
        print("\n   Sample match:")
        print(f"   Key: {sample_label.get('key')}")
        print(f"   Content: {sample_label.get('content', '')[:100]}...")
        
        gt = sample_label.get('ground_truth', {})
        print(f"   -> Matched to: {gt.get('title', 'N/A')[:60]}...")
        print(f"   -> Score: {gt.get('match_score', 'N/A')}")
else:
    print("⚠️  labels.json chưa được tạo (có thể do thiếu references.json)")

🏷️  labels.json: 33 matched references

   Sample match:
   Key: ref_2
   Content: @article{Kulik2006PRL,
pages = {103001},
year = {2006},
month = {September},
author = {Kulik, Heathe...
   -> Matched to: Density functional theory in transition-metal chemistry: a s...
   -> Score: 0.823


---
## 6. Chạy Full Pipeline (One-liner)

Nếu muốn chạy cả 2 phases cùng lúc, sử dụng `run_full_pipeline()`.

In [12]:
# Định nghĩa output mới để không ghi đè kết quả trước
DATA_OUTPUT_FULL = os.path.join(PROJECT_ROOT, 'data_output_full_demo')

print(f"📂 Output: {DATA_OUTPUT_FULL}")

📂 Output: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\data_output_full_demo


In [13]:
# Chạy full pipeline với 1 lệnh duy nhất
result = run_full_pipeline(
    data_raw=DATA_RAW,
    data_output=DATA_OUTPUT_FULL,
    parallel=True,
    max_workers=MAX_WORKERS,
    run_matching=True,  # Có chạy matching không
    verbose=True        # In thông tin tiến trình
)

print("\n" + "=" * 60)
print("📊 KẾT QUẢ TỔNG HỢP:")
print(f"   Papers processed: {result['processed']}")
print(f"   Papers matched:   {result['matched']}")
print(f"   Output path:      {result['output_path']}")

🚀 PHASE 1: Pre-processing & Parsing

✅ Phase 1 Complete: 11 papers processed

🔍 PHASE 2: Reference Matching
🚀 Starting Matching Pipeline (Phase 2.2)...
   Target: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\data_output_full_demo


Matching References: 100%|██████████| 11/11 [00:01<00:00,  7.35it/s]

✅ Matching Complete. 'labels.json' generated in all folders.

✅ Phase 2 Complete: 8 papers matched

🎉 PIPELINE COMPLETE!
   Output: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\data_output_full_demo

📊 KẾT QUẢ TỔNG HỢP:
   Papers processed: 11
   Papers matched:   8
   Output path:      d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\data_output_full_demo


## 📝 Ghi chú thêm

### Chạy từ Command Line
```bash
# Từ thư mục 23127011/
cd Milestone2_Project/23127011

# Full pipeline
python -m src.main --raw ../data_raw --output ../data_output --parallel

# Chỉ processing
python -m src.main --raw ../data_raw --output ../data_output --no-matching

# Chỉ matching

python -m src.main --output ../data_output --matching-only
```

### Xử lý lỗi thường gặp

1. **ModuleNotFoundError**: Đảm bảo đường dẫn `SRC_PARENT` đúng và đã thêm vào `sys.path`
2. **FileNotFoundError**: Kiểm tra lại đường dẫn `DATA_RAW`
3. **Empty output**: Kiểm tra file log trong `DATA_OUTPUT/pipeline.log`

### Cấu trúc data_raw

```bash
data_raw/
├── 2403-00530/
│   ├── tex/
│   │   ├── v1/
│   │   │   ├── main.tex
│   │   │   └── refs.bib
│   │   └── v2/
│   │       └── main.tex
│   ├── metadata.json
│   └── references.json
```